In [33]:
import pandas as pd
import glob
import os

# Data converter

- Data comes from XL in three formats, and put into the folder excel_data.

- Data can also come from csv's from csv_folder

- All data is converted to the right formatting as a csv inside final_etf_data

In [34]:
etf_files_path = 'final_etf_csv'

## Converts all XL data to correctly format csv files

In [35]:
excel_files_path = 'excel_data/format_'
num_formats = 3

In [36]:
format_1 = [
    'GlobalPE.xlsx',
    'STOXX600.xlsx'
]

format_2 = [
    'FTSE EMU Government Bond (EGBI) TRI.xlsx',
    'FTSECORP.xlsx',
    'FTSECORPALT.xlsx',
    'FTSEGOV.xlsx',
    'FTWBIGUSDT.xlsx'
    
]

format_3 = [
    'FTSE EPRA Nareit Developed Europe TRI.xlsx',
    'MSCI World.xlsx',
    'PE.xlsx',
    'FTSE EPRA Nareit Developed Europe.xlsx',
    'LPX50PE.xlsx',
    'MSCI ACWI.xlsx',
    'MSCI EUROPE.xlsx',
    'MSCI EMU.xlsx',
    'LPX50 Eur TR.xlsx',
    'MSCI World Net Index EUR.xlsx'
]

In [37]:
# Deletes every csv in final_etf_csv
files = glob.glob(os.path.join(etf_files_path, '*'))

for file in files:
    try:
        os.remove(file)
        print(f"Deleted: {file}")
    except Exception as e:
        print(f"Error deleting {file}: {e}")

Deleted: final_etf_csv\EURIBOR 1 Month.csv
Deleted: final_etf_csv\FTSE EMU Government Bond (EGBI) TRI.csv
Deleted: final_etf_csv\FTSE EPRA Nareit Developed Europe TRI.csv
Deleted: final_etf_csv\FTSE EPRA Nareit Developed Europe.csv
Deleted: final_etf_csv\FTSECORP.csv
Deleted: final_etf_csv\FTSECORPALT.csv
Deleted: final_etf_csv\FTWBIGUSDT.csv
Deleted: final_etf_csv\GlobalPE.csv
Deleted: final_etf_csv\ICE BofA EBL0 (Euro Large Cap Corporate).csv
Deleted: final_etf_csv\ICE BofA ER00 (Euro Corporate).csv
Deleted: final_etf_csv\ICE BofA G0BC (Corporate World).csv
Deleted: final_etf_csv\ICE BofA HE00 (Euro HY).csv
Deleted: final_etf_csv\ICE BofA HW00 (Global HY).csv
Deleted: final_etf_csv\LPX50 Eur TR.csv
Deleted: final_etf_csv\LPX50PE.csv
Deleted: final_etf_csv\MSCI ACWI.csv
Deleted: final_etf_csv\MSCI EMU.csv
Deleted: final_etf_csv\MSCI World 2.csv
Deleted: final_etf_csv\MSCI World Net Index EUR.csv
Deleted: final_etf_csv\MSCI World.csv
Deleted: final_etf_csv\PE.csv
Deleted: final_etf_csv

In [38]:
for i in range(1, 1 + num_formats):
    formated_files_path = excel_files_path + str(i)
    for filename in os.listdir(formated_files_path):
        if filename.endswith(".xlsx"):
            file_path = os.path.join(formated_files_path, filename)

            if filename in format_1:
                df = pd.read_excel(file_path, header=None)
                keyword_row_index = df[df.iloc[:, 0] == 'Exchange Date'].index[0]

                df = pd.read_excel(file_path, skiprows=keyword_row_index)
                df = df[['Exchange Date', 'Close']]
                df.dropna(subset=['Exchange Date', 'Close'], inplace=True)
                df.rename(columns={'Exchange Date' : 'Date', 'Close': 'Price'}, inplace=True)

            elif filename in format_2:
                df = pd.read_excel(file_path, skiprows=10)  
                df = df[['Date', 'Trade Price']]
                df.dropna(subset=['Date', 'Trade Price'], inplace=True)
                df.rename(columns={'Trade Price': 'Price'}, inplace=True)

            elif filename in format_3:
                df = pd.read_excel(file_path, skiprows=17)
                df = df[['Exchange Date', 'Close']]
                df.dropna(subset=['Exchange Date', 'Close'], inplace=True)
                df.rename(columns={'Exchange Date' : 'Date', 'Close': 'Price'}, inplace=True)
            
            else:
                continue
            
            csv_file_path = os.path.join(etf_files_path, f"{os.path.splitext(filename)[0]}.csv")
            
            # Save to CSV
            df.to_csv(csv_file_path, index=False)
            
            print(f"Converted {filename} to CSV and saved as {csv_file_path}")

Converted GlobalPE.xlsx to CSV and saved as final_etf_csv\GlobalPE.csv
Converted STOXX600.xlsx to CSV and saved as final_etf_csv\STOXX600.csv
Converted FTSE EMU Government Bond (EGBI) TRI.xlsx to CSV and saved as final_etf_csv\FTSE EMU Government Bond (EGBI) TRI.csv
Converted FTSECORP.xlsx to CSV and saved as final_etf_csv\FTSECORP.csv
Converted FTSECORPALT.xlsx to CSV and saved as final_etf_csv\FTSECORPALT.csv
Converted FTWBIGUSDT.xlsx to CSV and saved as final_etf_csv\FTWBIGUSDT.csv
Converted FTSE EPRA Nareit Developed Europe TRI.xlsx to CSV and saved as final_etf_csv\FTSE EPRA Nareit Developed Europe TRI.csv
Converted FTSE EPRA Nareit Developed Europe.xlsx to CSV and saved as final_etf_csv\FTSE EPRA Nareit Developed Europe.csv
Converted LPX50 Eur TR.xlsx to CSV and saved as final_etf_csv\LPX50 Eur TR.csv
Converted LPX50PE.xlsx to CSV and saved as final_etf_csv\LPX50PE.csv
Converted MSCI ACWI.xlsx to CSV and saved as final_etf_csv\MSCI ACWI.csv
Converted MSCI EMU.xlsx to CSV and save

## Converts imported CSVs into correct format in the right folder

In [39]:
csv_file_path = 'csv_data'

In [40]:
csv_files = [
    'ICE BofA EBL0 (Euro Large Cap Corporate).csv',
    'ICE BofA ER00 (Euro Corporate).csv',
    'ICE BofA G0BC (Corporate World).csv',
    'ICE BofA HE00 (Euro HY).csv',
    'ICE BofA HW00 (Global HY).csv',
    'MSCI World 2.csv',
    'EURIBOR 1 Month.csv'
]

In [41]:
for file in os.listdir(csv_file_path):
    if file.endswith('.csv'):
        file_path = os.path.join(csv_file_path, file)
        if file in csv_files:
            with open(file_path, 'r', encoding='utf-8') as f:
                first_line = f.readline().strip()
            
            if first_line.startswith("sep="):
                sep = first_line.split('=')[1]
                df = pd.read_csv(file_path, sep=sep, skiprows=1)
            else:
                df = pd.read_csv(file_path, sep=';')

            # Process Format 1: ["Date"; "Line(Q.MERER00)"]
            if df.columns[0] == 'Date' and len(df.columns) == 2:
                df.columns = ['Date', 'Price']
                df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y', errors='coerce')
                
                df['Price'] = df['Price'].apply(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)
                df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

            # Process Format 2: ["Date;Open;High;Low;Last;Volume;Status"]
            elif 'Date' in df.columns and 'Last' in df.columns:
                df = df[['Date', 'Open']]
                df.rename(columns={'Open': 'Price'}, inplace=True)
                df.columns = ['Date', 'Price']
                df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y', errors='coerce')
                
                df['Price'] = df['Price'].apply(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)
                df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
            
            elif 'Date' in df.columns and 'Last(QEURIBOR1MD=)' in df.columns:
                df = df[['Date', 'Last(QEURIBOR1MD=)']]
                df.rename(columns={'Last(QEURIBOR1MD=)': 'Price'}, inplace=True)
                df.columns = ['Date', 'Price']
                df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y', errors='coerce')
                
                df['Price'] = df['Price'].apply(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)
                df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

            else:
                print(f"Skipping unrecognized format in file: {file}")
                continue

            df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

            output_file_path = os.path.join(etf_files_path, file)
            df.to_csv(output_file_path, index=False, sep=',', header=['Date', 'Price'])
            print(f"Processed and saved: {output_file_path}")

Processed and saved: final_etf_csv\EURIBOR 1 Month.csv
Processed and saved: final_etf_csv\ICE BofA EBL0 (Euro Large Cap Corporate).csv
Processed and saved: final_etf_csv\ICE BofA ER00 (Euro Corporate).csv
Processed and saved: final_etf_csv\ICE BofA G0BC (Corporate World).csv
Processed and saved: final_etf_csv\ICE BofA HE00 (Euro HY).csv
Processed and saved: final_etf_csv\ICE BofA HW00 (Global HY).csv
Processed and saved: final_etf_csv\MSCI World 2.csv


## Handling EONIA data

In [42]:
file_path = "eonia_data/EONIA.xls"

df = pd.read_excel(file_path, skiprows=10)
df = df[['observation_date', 'EONIARATE']]
df.dropna(subset=['observation_date', 'EONIARATE'], inplace=True)
df.rename(columns={'observation_date' : 'Date', 'EONIARATE': 'Rate'}, inplace=True)
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

In [43]:
# Reversing
df = df.iloc[::-1]

In [44]:
df.head()

,Date,Rate
5997,2021-12-29,-0.493
5996,2021-12-28,-0.490
5995,2021-12-27,-0.491
5994,2021-12-24,-0.495
5993,2021-12-23,-0.491


In [45]:
file_path = "eonia_data/STR.csv"
df_2 = pd.read_csv(file_path, sep=';', skiprows=1)
df_2 = df_2[['Date', 'Last(QEUROSTR=)']]
df_2.rename(columns={'Last(QEUROSTR=)': 'Rate'}, inplace=True)
df_2.columns = ['Date', 'Rate']
df_2['Date'] = pd.to_datetime(df_2['Date'], format='%d.%m.%Y', errors='coerce')
df_2['Rate'] = df_2['Rate'].apply(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)
df_2['Rate'] = pd.to_numeric(df_2['Rate'], errors='coerce')
df_2['Date'] = df_2['Date'].dt.strftime('%Y-%m-%d')

In [46]:
df_2

,Date,Rate
0,2024-11-14,3.165
1,2024-11-13,3.164
2,2024-11-12,3.163
3,2024-11-11,3.162
4,2024-11-08,3.163
...,...,...
1311,2019-10-07,-0.554
1312,2019-10-04,-0.553
1313,2019-10-03,-0.555
1314,2019-10-02,-0.551


In [47]:
df_2.tail()

,Date,Rate
1311,2019-10-07,-0.554
1312,2019-10-04,-0.553
1313,2019-10-03,-0.555
1314,2019-10-02,-0.551
1315,2019-10-01,-0.549


In [48]:
merged_df = pd.concat([df, df_2]).groupby('Date', as_index=False).last()
merged_df.sort_values('Date', inplace=True)
merged_df.reset_index(drop=True, inplace=True)

In [49]:
# Reversing
merged_df = merged_df.iloc[::-1]

merged_df['Rate'] = merged_df['Rate'] / 100
merged_df['Rate'] = merged_df['Rate'] / 365

In [50]:
merged_df

,Date,Rate
6626,2024-11-14,0.000087
6625,2024-11-13,0.000087
6624,2024-11-12,0.000087
6623,2024-11-11,0.000087
6622,2024-11-08,0.000087
...,...,...
4,1999-01-08,0.000088
3,1999-01-07,0.000088
2,1999-01-06,0.000088
1,1999-01-05,0.000088


In [51]:
output_file_path = 'final_eonia/eonia_rate.csv'
merged_df.to_csv(output_file_path, index=False, sep=',', header=['Date', 'Rate'])